In [ ]:
from pymilvus import connections, utility, MilvusException, FieldSchema, CollectionSchema, DataType, Collection, loading_progress
connections.connect(host="localhost", port="19530", db_name='default')

In [ ]:
fields = [
    FieldSchema(name="Data_Id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="Dataset_File", dtype=DataType.VARCHAR,max_length=255),
    FieldSchema(name="Collected", dtype=DataType.VARCHAR,max_length=100),
    FieldSchema(name="Variable", dtype=DataType.VARCHAR,max_length=100),
    FieldSchema(name="Label", dtype=DataType.VARCHAR,max_length=255),
    FieldSchema(name="Description", dtype=DataType.VARCHAR,max_length=1024),
    FieldSchema(name="Info_embedding", dtype=DataType.FLOAT_VECTOR, dim=768),
]

collection_schema = CollectionSchema(fields, description="Example_collection")

doc_data = Collection("Milvus_Test_Sentrans", collection_schema)


In [ ]:
#Index on vector
def Create_Index(collection_Name: str, vector_name: str, index_par: dict):
    try:
        collection = Collection(collection_Name)
        collection.create_index(field_name = vector_name, index_params = index_par)
    except Exception as e:
        print(e)
    else:
        print("Index created")

index_params = {
    "index_type" : "IVF_FLAT",
    "metric_type" : "L2",
    "params" : {"nlist":128}  ,
}

Create_Index("Milvus_Test_Sentrans", "Info_embedding", index_params) 


In [ ]:
Collection_Name = 'Milvus_Test_Sentrans'
collection = Collection(Collection_Name)
collection.load()

In [ ]:
#insert function 
#NOT TEST YET
def Insert_vector(id_list: list, data_file_list: list, collected_list: list, variable_list: list, label_list: list, Description_list: list, embeddings_Description, Collection):
    try:
        collection = Collection
 
        for id_item, data_file_item, collected_item, variable_item, label_item, embeddings_Description in zip(id_list, data_file_list, collected_list, variable_list, label_list, embeddings_Description):
            entities = [[id_item], [data_file_item], [collected_item], [variable_item], [label_item], [embeddings_Description]]
            collection.insert(entities)
            print(f"编号{id_item}插入完毕")
        collection.flush()
 
    except Exception as e:
        print(e)

In [ ]:
def Insert_vector2(id_list: list, data_file_list: list, collected_list: list, variable_list: list, label_list: list, Description_list: list, embeddings_Description, Collection):
    try:
        collection = Collection
 
        for id in id_list:
            index = id - 1
            entities = [[id_list[index]], [data_file_list[index]], [collected_list[index]], [variable_list[index]], [label_list[index]], [Description_list[index]], [embeddings_Description[index]]]
            collection.insert(entities)
            print(f"编号{id}插入完毕")
    except Exception as e:
        print(e)

In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# nltk.download('wordnet')
# nltk.download('omw-1.4')  
# nltk.download('punkt')

from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

def nltk_tokenize(text):
    return word_tokenize(text)
    
def remove_stopwords(text):

    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

def lemmatize_text(text):
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
    return ' '.join(lemmatized_text)
    
def stem_tokens(token_list):
    word_tokens = word_tokenize(text)
    
    return [stemmer.stem(token) for token in token_list]


In [ ]:
import time
import pandas as pd
import json

from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import model

sentence_transformer_ef = model.dense.SentenceTransformerEmbeddingFunction(
    model_name='all-distilroberta-v1', # Specify the model name
    device='cpu' # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)

Collection_Name = 'Milvus_Test_Sentrans'
collection = Collection(Collection_Name)

file_path = 'data_WHI.csv'
df = pd.read_csv(file_path)

id_list = df.index.tolist()
id_plus_one = [x + 1 for x in id_list]

dataset_file_data = df["Dataset File"].tolist()
collected_data = df.Collected.tolist()
variable_data = df.Variable.tolist()



df['Label'] = df['Label'].fillna(' ')
label_data = df['Label'].tolist()

df['Description'] = df['Description'].fillna(' ')
description_data = df['Description'].tolist()

df['combine'] = df['Label'] + df['Description']
df['combine'] = df['combine'].str.replace('[^a-zA-Z\s]', '', regex=True)
df['combine'] = df['combine'].str.lower()
df['combine_no_stopwords'] = df['combine'].apply(remove_stopwords)
df['lemmatize_tokens'] = df['combine_no_stopwords'].apply(lemmatize_text)
combine_data = df['lemmatize_tokens'].tolist()
# import ipdb;ipdb.set_trace()

#embedding
Info_embedding = sentence_transformer_ef.encode_documents(combine_data)


# import ipdb;ipdb.set_trace()


start_time = time.time()
try:
    Insert_vector2(id_plus_one, dataset_file_data, collected_data, variable_data, label_data, description_data, Info_embedding, Collection = collection)
    
except Exception as e:
    print(f"插入数据时发生错误：{e}")
end_time = time.time()
print(f"插入数据用时为: {end_time - start_time}秒, 平均一条的插入时间为: {(end_time - start_time)/len(id_list)}")
print(f"总共有: {len(id_list)}条数据")